In [22]:
import os
import lxml
import re
import bs4
import numpy as np
import glob
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
from IPython.core.display import display, HTML


<h3> Creating necessary function to find the data

In [14]:
def extract_text(title_):
    """extract text for a given title from a beautiful soup object """
        
    try:
        text=soup.find(text=title_).parent.find_next_siblings()[0]
        text = [td.text for td in text('td')]
        text = np.unique(text)
        
    except:
        text = None
    
    return title_, text

def time_log_start():
    import datetime
    now = datetime.datetime.now()
    print("Start date and time :")
    print(now.strftime("%Y-%m-%d %H:%M:%S"))
    
    
def time_log_finish():
    import datetime
    now = datetime.datetime.now()
    print("Finish date and time :")
    print(now.strftime("%Y-%m-%d %H:%M:%S"))


<h3> Importing the data, creating a dataframe with all fields.

In [152]:
df_fin = pd.DataFrame(columns=['Filename','Additional Source Comments','Advance Directives', 'Allergies', 
                               'Assessment', 'Chief Complaint','Coordinator For Healthplans - Encounter Summary',
                               'Family History', 'Functional Status',
                               'History of Present Illness', 'Immunizations',
                               'Instructions','Interventions','Medical Equipment', 'Medications',
                               'Mental Status','Medications Administered', 'Past Encounters','Patient Summarization (C-CDA 2.1 CCD)',
                               'Past Medical History', 'Payers', 'Physical Exam', 'Plan of Care','Plan of Treatment'
                               'Problems', 'Procedures', 'Reason for Visit', 'Results',
                               'Review of Systems', 'Review of systems Narrative - Reported','Social History',
                               'Vaccine List', 'Vitals','Vital Signs','History of Past illness Narrative',
                               'Chief complaint Narrative - Reported','Reason for visit Narrative'])



<h3> Import the CCD xml data

In [5]:
fp_1 = '/mapr/datalake/optum/optuminsight/opsirsch1/dsar/CCD'
os.chdir(fp_1)
list_xml_all = glob.glob("*.xml")
print('Number of files =',len(list_xml_all))
print(list_xml_all[0:10])


Number of files = 940
['MEBMRVXG_athena_8359333851_20732_ccda_3.xml', 'MEC0785000053274800_athena_8201075755_3659_ccda_4.xml', 'MEBLW7YR_athena_8359121423_20664_ccda_1.xml', 'MEBMRVXG_athena_8359333851_20732_ccda_2.xml', 'MEBMRVXG_athena_8359333851_20732_ccda_0.xml', 'MEC0785000103997800_athena_8200897169_9872_ccda_1.xml', 'MEBMMS7Y_athena_8359327503_20732_ccda_3.xml', 'MSP7302192233663500_athena_8359183495_3659_ccda_2.xml', 'MEBGKPRQ_athena_8359345385_20732_ccda_1.xml', 'MEBMG86K_athena_8359336654_20732_ccda_3.xml']


<h3> Reading in the CCD xml data and performing data transformations

In [164]:
time_log_start()
content = []
n = len(list_xml_all)
# Read the XML file
for i in range(0,n):
    with open(list_xml_all[i], "r") as file:
        content = file.readlines()
        content = "".join(content)
        soup = bs(content, "lxml")
    
          
        ###Getting tuple to list
        fn = [("Filename",list_xml_all[i])]
        titles = np.unique(soup.find_all('title'))  ##jeff


        for title_ in titles:
            result = extract_text(title_)
            out = [result]
            out1 = [item for t in out for item in t]
    #Getting each file and creating a dataframe, transform or edit some of the data
            fn.extend(out)
    
            df = pd.DataFrame(fn)
            df.fillna('', inplace=True)
            
            df[1] = [''.join(map(str, l)) for l in df[1]]
            df[1] = df[1].str.replace("\n"," ")
            df[1] = df[1].str.replace("<p>","")
            df[1] = df[1].str.lower()
        
            df_0 = df.reset_index(drop=True)
            df_0t = df_0.transpose()
            header_row = 0
            df_0t.columns = df_0t.iloc[header_row]
            df_0t = df_0t.drop(header_row)
            
            d = df_0t.columns.tolist()
            new_columns = [re.sub('\d{4}\-\d{2}\-\d{2}', '', l).strip() for l in d]
            
            df_0t.columns = new_columns
            df_fin = df_fin.append(df_0t)
            
            
            
print(df_fin.shape) 

g = df_fin.groupby('Filename')
df_final=g.tail(1)
print(df_final.shape)
print(df_final.head(n=13))
print('===============')
time_log_finish()

Start date and time :
2021-03-05 13:28:55
(23707, 41)
(940, 41)
                                            Filename  \
1        mebmrvxg_athena_8359333851_20732_ccda_3.xml   
1  mec0785000053274800_athena_8201075755_3659_ccd...   
1        meblw7yr_athena_8359121423_20664_ccda_1.xml   
1        mebmrvxg_athena_8359333851_20732_ccda_2.xml   
1        mebmrvxg_athena_8359333851_20732_ccda_0.xml   
1  mec0785000103997800_athena_8200897169_9872_ccd...   
1        mebmms7y_athena_8359327503_20732_ccda_3.xml   
1  msp7302192233663500_athena_8359183495_3659_ccd...   
1        mebgkprq_athena_8359345385_20732_ccda_1.xml   
1        mebmg86k_athena_8359336654_20732_ccda_3.xml   
1  msp7302192233663500_athena_8183111771_3659_ccd...   
1  mec0785000169626300_athena_8359164966_9872_ccd...   
1  mec0785000261733500_athena_8201214433_9872_ccd...   

                          Additional Source Comments Advance Directives  \
1                                  medication refill                      
1

In [166]:

df_final.tail()

,Filename,Additional Source Comments,Advance Directives,Allergies,Assessment,Chief Complaint,Coordinator For Healthplans - Encounter Summary,Family History,Functional Status,History of Present Illness,...,Vitals,Vital Signs,History of Past illness Narrative,Chief complaint Narrative - Reported,Reason for visit Narrative,Encounters,Goals,History of Present illness Narrative,Plan of Treatment,Problems
1,mebmj1ch_athena_8359335958_20732_ccda_0.xml,fall; pneumonia; hospital f/u,,drug allergysc - metrolina medical associates-...,NaN,fall; pneumonia; hospital f/u,NaN,(no notes) (no information)asthmabrotherdiabe...,,NaN,...,NaN,11-24-2020 12:00-0500132 mm[hg]154.94 cm76 /mi...,NaN,NaN,NaN,sc - metrolina medical associates- - main offi...,,note:pt is here for hospital follow up. he w...,medicare physicalsc - metrolina medical associ...,prostate- 2012 abnormal gaitaccidental fallac...
1,mec0787900172384400_athena_8201038867_3659_ccd...,onychomycosis due to dermatophyte; diabetic fo...,,nkda - no known drug allergy â,NaN,onychomycosis due to dermatophyte; diabetic fo...,NaN,(no notes) (no information)fatherheart diseas...,,NaN,...,NaN,05-21-2020 12:00-0400177.8 cm26.5 kg/m283.92 k...,NaN,NaN,NaN,kari elizabeth prescott dpm ...,,â foot- podiatry reported by: pati...,"follow upmn - kari e prescott, d.p.m., p.a.,of...",neuropathy acquired keratoderma; translations...
1,mec0785000104959300_athena_8359142704_9872_ccd...,diabetic foot care,,nkda - no known drug allergy â,NaN,diabetic foot care,NaN,NaN,,NaN,...,NaN,02-13-2017 12:00-0500160.02 cm25.9 kg/m266.23 ...,NaN,NaN,NaN,mn - northland foot and ankle specialist - hig...,,note:this is the initial clinic visit for th...,appointments imaging lab procedures refer...,aids/hivacid reflux/gastritisacute cerebrovasc...
1,mec0785000081924900_athena_8201093394_9872_ccd...,routine foot care,,nkda - no known drug allergy â,NaN,routine foot care,NaN,NaN,,NaN,...,NaN,02-15-2017 12:00-0500113 mm[hg]167.64 cm23.2 k...,NaN,NaN,NaN,mn - northland foot and ankle specialist - hig...,,â routine foot care reported by: p...,appointments imaging lab procedures refer...,aids/hivacid reflux/gastritisacute cerebrovasc...
1,mec0787900172384400_athena_8183216807_3659_ccd...,bilateral diabetic foot care,,nkda - no known drug allergy â,NaN,bilateral diabetic foot care,NaN,(no notes) (no information)fatherheart diseas...,,NaN,...,NaN,11-03-2020 12:00-0500177.8 cm26.5 kg/m283.92 k...,NaN,NaN,NaN,kari elizabeth prescott dpm ...,,â foot- podiatry reported by: pati...,"follow upmn - kari e prescott, d.p.m., p.a.,of...",neuropathy acquired keratoderma; translations...


In [530]:
df_final.to_parquet('pilot.parquet')
print(os.getcwd())
#os.listdir()


/mapr/datalake/optum/optuminsight/opsirsch1/dsar/CCD
